In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np

# Define image dimensions
img_width, img_height = 150, 150

# Load data
train_data_dir = "drive/My Drive/healthy_training_set"
validation_data_dir = "drive/My Drive/validation"

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode="categorical"
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode="categorical"
)

# Define the CNN model
model = tf.keras.models.Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(11, activation="softmax"),  # Number of classes
])

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=70,  # Use the actual number of batches per epoch
    epochs=10,
    validation_data=validation_generator,
    validation_steps=4
)

# Save the model
model.save("plant_identifier.h5")

# Use the trained model to identify new plants
plant_image = tf.keras.preprocessing.image.load_img("drive/My Drive/test_images/test.jpg", target_size=(img_width, img_height))
plant_image = tf.keras.preprocessing.image.img_to_array(plant_image)
plant_image = plant_image / 255.0
plant_image = np.expand_dims(plant_image, axis=0)

# Create a dictionary mapping class indices to their names using directory names
class_names = {}
for i, class_name in enumerate(train_generator.class_indices.keys()):
    class_names[i] = class_name

prediction = model.predict(plant_image)
plant_class = np.argmax(prediction)

print("Predicted plant:", class_names[plant_class])


Found 2167 images belonging to 11 classes.
Found 110 images belonging to 11 classes.
Epoch 1/10
 8/70 [==>...........................] - ETA: 18:29 - loss: 2.8318 - accuracy: 0.1133

# New Section